In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup

In [2]:
page = wptools.page('Bachir_Gemayel')
page.get_parse()

en.wikipedia.org (parse) Bachir_Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...
Bachir Gemayel (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Bachir Wi...
  infobox: <dict(25)> name, image, office, status, term_label, ter...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:B...
  pageid: 23662554
  parsetree: <str(44998)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Bachir Gemayel
  wikibase: Q315534
  wikidata_url: https://www.wikidata.org/wiki/Q315534
  wikitext: <str(37059)> {{Short description|Lebanese politician a...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Bachir_Gemayel",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Bachir Gemayel')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1982 Lebanon War
Achrafieh
Ain El Remmaneh
Al-Tanzim
Al Jazeera
Alfred Naqqache
Amine Gemayel
Arabic language
Ariel Sharon
As-Saiqa
Ayoub Tabet
Battle of Zahleh
Battle of the Hotels
Bechara El Khoury
Beirut
Beqaa Governorate
Bikfaya
Black Saturday (Lebanon)
Bsharri
Camille Chamoun
Chamoun family
Charles Debbas
Charles Helou
Collège Notre Dame de Jamhour
Constitutional Bloc (Lebanon)
Dallas
Damour
Dany Chamoun
Doi (identifier)
Don Bosco
Ehden
Ehden massacre
Elias Hrawi
Elie Hobeika
Fadi Frem
Federal Bureau of Investigation
Fouad Abou Nader
Fouad Siniora
Frangieh family
Free Patriotic Movement
Fuad Chehab
Gebran Bassil
Gemayel family
Geneviève Gemayel
George Hawi
Greater Lebanon
Habib Pacha Es-Saad
Habib Shartouni
Hafez al-Assad
Hamra Street
Hani al-Hassan
Hundred Days' War
ISBN (identifier)
ISNI (identifier)
Internal Security Forces
Israel
Israeli Defense Force
Kamal Jumblatt
Karim Pakradouni
Kataeb Party
Kataeb Regulatory Forces
Keserwan District
Lawyer
Lebanese Army
Lebanese Ci

In [4]:
def get_arabic_name(english_phrase):
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s




In [5]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [7]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    if isinstance(name, list):
                        if len(name) > 1:
                            for n in name:
                                print('name before: ', n)
                                name = [n]
                                break
#                             name = list(name[0])
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) Achrafieh
en.wikipedia.org (parse) Ain El Remmaneh
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Al Jazeera
en.wikipedia.org (parse) Alfred Naqqache
en.wikipedia.org (imageinfo) File:Alfred Naqqache 2.jpg
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon Headshot.jpg
en.wikipedia.org (parse) As-Saiqa


-----------------
name:  ['As-Saiqa']
founders:  
ideology:  ['Pan-Arabism', 'Baathism']
position:  -
START
name:  As-Saiqa
name ar:  منظمة الصاعقة 
name ar link:  https://ar.wikipedia.org/wiki/%D9%85%D9%86%D8%B8%D9%85%D8%A9_%D8%A7%D9%84%D8%B5%D8%A7%D8%B9%D9%82%D8%A9
END
START
ideology:  Pan-Arabism
ideology ar:  وحدة عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%88%D8%AD%D8%AF%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Baathism
ideology ar:  بعثية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D8%B9%D8%AB%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:Ayoub Thabit.png
en.wikipedia.org (parse) Battle of Zahleh
en.wikipedia.org (imageinfo) File:Zahle,Lebanon.JPG
en.wikipedia.org (parse) Battle of the Hotels
en.wikipedia.org (imageinfo) File:Holiday Inn Beirut 4221308304 3...
en.wikipedia.org (parse) Bechara El Khoury
en.wikipedia.org (imageinfo) File:Bechara El Khoury - 1947.jpg
en.wikipedia.org (parse) Beirut
en.wikipedia.org (parse) Beqaa Governorate
en.wikipedia.org (parse) Bikfaya
en.wikipedia.org (parse) Black Saturday (Lebanon)
en.wikipedia.org (parse) Bsharri
en.wikipedia.org (parse) Camille Chamoun
en.wikipedia.org (imageinfo) File:Camille chamoun.jpg
en.wikipedia.org (parse) Chamoun family
en.wikipedia.org (parse) Charles Debbas
en.wikipedia.org (imageinfo) File:دباس.jpg
en.wikipedia.org (parse) Charles Helou
en.wikipedia.org (imageinfo) File:Charles Helou.jpg
en.wikipedia.org (parse) Collège Notre Dame de Jamhour
en.wikipedia.org (parse) Constitutional Bl

-----------------
name:  ['Constitutional Bloc']
founders:  
ideology:  ['Nonsectarianism', 'Pro-National Pact', 'Liberal nationalism']
position:  ['Centre, Secular']
START
name:  Constitutional Bloc
name ar:  الكتلة الدستورية (مترجمه)
name ar link:  -1
END
START
ideology:  Nonsectarianism
ideology ar:  اللاطائفية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%84%D8%A7%D8%B7%D8%A7%D8%A6%D9%81%D9%8A%D8%A9
Cannot find the link: http://en.wikipedia.org/wiki/Pro-National Pact
ideology:  Pro-National Pact
ideology ar:  الميثاق المؤيد للوطن (مترجمه)
ideo ar link:  -1
ideology:  Liberal nationalism
ideology ar:  قومية مدنية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%85%D8%AF%D9%86%D9%8A%D8%A9
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Centre, Secular
position:  Centre, Secular
position ar:  مركز علماني (مترجمه)
position ar link:  -1
END
START
END


en.wikipedia.org (parse) Dallas
en.wikipedia.org (parse) Damour
en.wikipedia.org (parse) Dany Chamoun


KeyboardInterrupt: 

In [8]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,As-Saiqa,"Pan-Arabism,Baathism",-,
1,Constitutional Bloc,"Nonsectarianism,Pro-National Pact,Liberal nati...","Centre, Secular",


In [9]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,منظمة الصاعقة,"وحدة عربية ,بعثية",-,
1,الكتلة الدستورية (مترجمه),"اللاطائفية ,الميثاق المؤيد للوطن (مترجمه),قومي...",مركز علماني (مترجمه),


In [10]:
datasets_dir = 'datasets_updated/1982-1984/Bachir_Gemayel_Assasination/'

In [11]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [12]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [13]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) Achrafieh
en.wikipedia.org (parse) Ain El Remmaneh
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Al Jazeera
en.wikipedia.org (parse) Alfred Naqqache
en.wikipedia.org (imageinfo) File:Alfred Naqqache 2.jpg


-----------------
name:  ['Alfred Georges Naqqache']
native_name:  -
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  -
name ar:  ألفرد نقاش 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
name ar:  ألفرد نقاش 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  -


en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg


-----------------
name:  ['Amine Gemayel']
native_name:  ['أمين الجميل']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  ['Lebanese']
name ar:  أمين الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  أمين الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon Headshot.jpg


-----------------
name:  -
native_name:  -
nickname:  -
religion:  -
party:  ['plainlist']
nationality:  -
name ar:  أرئيل شارون 
START
Cannot find the link: http://en.wikipedia.org/wiki/plainlist
political party:  plainlist
political party ar:  قائمة عادية (مترجمه)
political party ar link:  -1
END
name ar:  أرئيل شارون 
religion ar:  ['قائمة عادية (مترجمه)']
party ar:  ['plainlist']
nationality ar:  -


en.wikipedia.org (parse) As-Saiqa
en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:Ayoub Thabit.png


-----------------
name:  ['Ayoub Tabet', 'أيوب تابت']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  -
name ar:  أيوب ثابت 
name ar:  أيوب ثابت 
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Battle of Zahleh
en.wikipedia.org (imageinfo) File:Zahle,Lebanon.JPG
en.wikipedia.org (parse) Battle of the Hotels
en.wikipedia.org (imageinfo) File:Holiday Inn Beirut 4221308304 3...
en.wikipedia.org (parse) Bechara El Khoury
en.wikipedia.org (imageinfo) File:Bechara El Khoury - 1947.jpg


KeyboardInterrupt: 

In [14]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,"(Alfred Georges Naqqache,)",Kataeb Party,-,-
1,"(Amine Gemayel,)",Kataeb Party,Lebanese,-
2,"(-,)",plainlist,-,-
3,"(Ayoub Tabet,أيوب تابت,)",-,-,-


In [15]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,حزب الكتائب اللبنانية,ألفرد نقاش
1,-,لبناني (مترجمه),حزب الكتائب اللبنانية,أمين الجميل
2,-,-,قائمة عادية (مترجمه),أرئيل شارون
3,-,-,-,أيوب ثابت


In [16]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [17]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)
                    
                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'
    
                    df_locations_en = df_locations_en.append({
                        'Name': ','.join(name) if isinstance(name, list) and len(name) > 1 else name,
                        'Subdivision type': ','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type,
                        'Subdivision name': ','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name                    
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)
                    
                    
                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass
                    
                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'
        

                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = df_locations_ar.append({
                            'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                            'نوع التقسيم': ','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar,
                            'اسم التقسيم': ','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar,
                        }, ignore_index=True)
                    
                    else:
                        print('skipping {} because name is not found'.format(p))
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) Achrafieh


-----------------
name:  -
subdivision_type:  ['Country']
subdivision_name:  ['flagcountry', 'Lebanon']
skipping Achrafieh because name is not found


en.wikipedia.org (parse) Ain El Remmaneh
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Al Jazeera
en.wikipedia.org (parse) Alfred Naqqache
en.wikipedia.org (imageinfo) File:Alfred Naqqache 2.jpg
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon Headshot.jpg
en.wikipedia.org (parse) As-Saiqa
en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:Ayoub Thabit.png
en.wikipedia.org (parse) Battle of Zahleh


KeyboardInterrupt: 

In [19]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,-,Country,"flagcountry,Lebanon"


In [20]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم


In [21]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')